In [1]:
import pandas as pd

dataframes_df = pd.read_excel('../.noteook-tests/Overview-reference-data-phase-1-20231222.xlsx', sheet_name=None)
dataframes_df = dataframes_df['Overview']
dataframes_df

,Final data sets (name of metadata Excel docs),Name_Ewoc_II,Geoparquet Status,Dataset name,Metadata Status,Harmonization PDF Status,Ready for upload into RDM,Remark,count public labels,check public no,POLICY,DATATYPE,CIB3,Analysis,Problematic,in Access db,Update-WENR-June-2024,Location (names can be different from metadata Excel doc)
0,2016_AF_OAF_POINT_110,2016_AF_One-Acre-Fund-MEL_POINT_110,not required,Extra data to improve classification,not required,not required,not required,NaN,3780.0,1.0,public,FO,NaN,not in CIB3,"yes, too old",YES,DONE,/data/worldcereal/data/ref/WENR/One-acre-fund/...
1,2016_AS_Remelgado_POLY_111,2016_AS_CAWA-project_POLY_111,not required,CAWa project,not required,not required,not required,NaN,5214.0,1.0,public,FO,NaN,not in CIB3,"yes, too old",YES,DONE,/data/worldcereal/data/ref/WENR/Remelgado-Aral...
2,2016_BF_JECAM_CIRAD_POLY_111,2016_BFA_JECAM-CIRAD_POLY_111,not required,JECAM-CIRAD,not required,not required,not required,NaN,847.0,1.0,public,FO,NaN,not in CIB3,"yes, too old",YES,DONE,/data/worldcereal/data/ref/WENR/JECAM/CIRAD
3,2016_BR_JECAM_CIRAD_POLY_111,2016_BRA_JECAM-CIRAD_POLY_111,not required,JECAM-CIRAD,not required,not required,not required,NaN,2774.0,1.0,public,FO,NaN,not in CIB3,"yes, too old",YES,DONE,/data/worldcereal/data/ref/WENR/JECAM/CIRAD
4,2016_CAN_AAFC-ACIGTD_POINT_110,2016_CAN_AAFC-Crop-Inventory_POINT_110,not required,AAFC Crop Inventory,not required,not required,not required,NaN,101389.0,1.0,public,FO,NaN,not in CIB3,"yes, too old",YES,DONE,/data/worldcereal/data/ref/WENR/AAFC-Canada
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,Not to be included in Phase II,Not to be included in Phase II,not required,Extra data to improve classification,not required,not required,not required,NaN,NaN,NaN,NaN,NaN,2020_TEST_test_POINT_100,in CIB3,NaN,NaN,NaN,NaN
200,Not to be included in Phase II,Not to be included in Phase II,not required,Extra data to improve classification,not required,not required,not required,NaN,NaN,NaN,NaN,NaN,2021_GLO_irr-eleaf_POLY_101,in CIB3,NaN,NaN,NaN,NaN
201,Not to be included in Phase II,Not to be included in Phase II,not required,Extra data to improve classification,not required,not required,not required,NaN,NaN,NaN,NaN,NaN,2021_GLO_nat-eleaf_POLY_101,in CIB3,NaN,NaN,NaN,NaN
202,Not to be included in Phase II,2020_HRV_Eurocrops_POLY_110,not required,Extra data to improve classification,not required,not required,not required,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
# Datasets that are ready to be extracted and uploaded to RDM
to_process_dfs = dataframes_df[dataframes_df['Geoparquet Status'] == 'ready']['Name_Ewoc_II'].tolist()
len(to_process_dfs)

138

In [3]:
from pathlib import Path
from tqdm import tqdm

import geopandas as gpd
import warnings
warnings.simplefilter('ignore')

dataset_paths = [
    Path('/vitodata/worldcereal/data/RDM/') / name / 'harmonized' / (name + '.geoparquet')
    for name in to_process_dfs
]

geometry_stats = {}
for df_name, path in tqdm(zip(to_process_dfs, dataset_paths)):
    df = gpd.read_parquet(path)
    
    df["geometry"] = df.geometry.centroid
    median_latitude = abs(df['geometry'].y).median()    
    geometry_stats[df_name] = median_latitude

geometry_stats

138it [07:24,  3.22s/it]


{'2017_AF_One-Acre-Fund-MEL_POINT_110': 0.479936115369225,
 '2017_ARG_LISTA-field-data_POLY_110': 36.89348413209724,
 '2017_AS_CAWA-project_POLY_111': 40.503829274042864,
 '2017_AUT_LPIS_POLY_110': 47.84663619162726,
 '2017_BEL_LPIS-Flanders_POLY_110': 51.00915738448651,
 '2017_BFA_JECAM-CIRAD_POLY_111': 11.236742374748045,
 '2017_BRA_JECAM-CIRAD_POLY_111': 23.161398650681527,
 '2017_CAN_AAFC-Crop-Inventory_POINT_110': 44.873407769552955,
 '2017_CMR_CGIAR-GARDIAN_POINT_110': 4.90065,
 '2017_KEN_IIASA_POINT_100': 1.195521762651465,
 '2017_KEN_IIASA_POLY_100': 1.6738550741637401,
 '2017_LBN_FAO-WAPOR-1_POLY_111': 33.83326439383714,
 '2017_LBN_FAO-WAPOR-2_POLY_111': 33.83321885951537,
 '2017_LBN_FAO-WAPOR-3_POLY_111': 33.719963855344595,
 '2017_LBN_FAO-WAPOR-4_POLY_111': 33.72183417454755,
 '2017_MDG_JECAM-CIRAD_POLY_111': 19.67670380895113,
 '2017_NGA_CGIAR-GARDIAN_POINT_110': 10.017965,
 '2017_SSD_ESA-project-Sen2Agri_POLY_111': 6.402087402883581,
 '2017_SSD_ESA-project-Sen2Agri_POINT_1

In [5]:
# Save the reference dataset to a new parquet file
dest_folder = Path('/vitodata/worldcereal_data/EXTRACTIONS/all_datasets/')

def clear_write_dataset(ref_df, current_dataset_name, geometry_stats, dest_folder):
    ref_df['ref_id'] = current_dataset_name

    if " h3_l3_cell" in ref_df.columns:
        print("Rename column h3_l3_cell for dataset: ", current_dataset_name)
        ref_df = ref_df.rename(columns={" h3_l3_cell": "h3_l3_cell"})

    required_columns_and_types = [
        ('ref_id', 'str'),
        ('sample_id', 'str'),
        ('h3_l3_cell', 'str'),
        ('valid_time', 'str'),
        ('extract', 'int32'),
        ('ewoc_code', 'int64')
    ]

    for column, dtype in required_columns_and_types:
        if column not in ref_df.columns:
            raise ValueError(f'Column {column} not found in {current_dataset_name}')
        if ref_df[column].dtype != dtype:
            warnings.warn(f'Column {column} has dtype {ref_df[column].dtype} but should be {dtype}')
            try:
                ref_df[column] = ref_df[column].astype(dtype)
            except:
                print(f"Exception when trying to convert column {column} from dataset {current_dataset_name} to {dtype}")
                return

    median_latitude = str(int(round(geometry_stats[current_dataset_name]))).zfill(2)

    dest_file = dest_folder / f'{median_latitude}_{current_dataset_name}.geoparquet'
    ref_df.to_parquet(dest_file, index=False)


for current_dataset_path, current_dataset_name in tqdm(zip(dataset_paths, to_process_dfs)):
    if not current_dataset_path.exists():
        raise FileNotFoundError(f'Couldn\'t find {current_dataset_name} in path: {current_dataset_path}')
    try:
        ref_df = gpd.read_parquet(current_dataset_path)
    except Exception:
        raise IOError(f'Failed to read {current_dataset_path}')

    clear_write_dataset(ref_df, current_dataset_name, geometry_stats, dest_folder)


23it [01:35,  5.40it/s]

Exception when trying to convert column extract from dataset 2017_TZA_OSF-AFSIS_POINT_110 to int32


48it [03:32,  1.75s/it]

Rename column h3_l3_cell for dataset:  2018_TZA_OSF-AFSIS_POINT_110
Exception when trying to convert column extract from dataset 2018_TZA_OSF-AFSIS_POINT_110 to int32


55it [03:33,  2.76it/s]

...

Rename column h3_l3_cell for dataset:  2019_TZA_One-Acre-Fund-MEL_POINT_110
Exception when trying to convert column extract from dataset 2019_TZA_One-Acre-Fund-MEL_POINT_110 to int32


107it [10:32,  3.13s/it]

Rename column h3_l3_cell for dataset:  2020_RWA_FAO-WAPOR-1_POINT_111
Exception when trying to convert column extract from dataset 2020_RWA_FAO-WAPOR-1_POINT_111 to int32


138it [15:23,  6.69s/it]


: 